In [1]:
from tensorflow.keras.layers import Input, Embedding, LSTM, TimeDistributed, Dense, Concatenate, Bidirectional
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.preprocessing.sequence import pad_sequences
from BahdanauAttention import AttentionLayer
import numpy as np
import random
import eng_to_ipa as ipa
import pickle

In [2]:
class bi_model:
    def __init__(self, max_encoder_len, max_decoder_len, build_inference_model_encoder_vocab, num_decoder_vocab):
        self.latent_dim = 256
        self.embedding_dim = 200
        self.max_encoder_len = max_encoder_len
        self.max_decoder_len = max_decoder_len
        self.num_encoder_vocab = num_encoder_vocab
        self.num_decoder_vocab = num_decoder_vocab

        self.build_encoder()
        self.build_decoder()
        self.training_model = Model([self.encoder_inputs, self.decoder_inputs], self.decoder_outputs) 
        
    def build_encoder(self):
        self.encoder_inputs = Input(shape=(self.max_encoder_len,)) 
        # Embedding layer- i am using 1024 output-dim for embedding you can try diff values 100,256,512,1000
        self.enc_emb = Embedding(self.num_encoder_vocab, self.embedding_dim, trainable = True)(self.encoder_inputs)

        # Bidirectional lstm layer
        self.enc_lstm1 = Bidirectional(LSTM(self.latent_dim,return_sequences=True,return_state=True))
        self.encoder_outputs1, self.forw_state_h, self.forw_state_c, self.back_state_h, self.back_state_c = self.enc_lstm1(self.enc_emb)

        # Concatenate both h and c 
        self.final_enc_h = Concatenate()([self.forw_state_h,self.back_state_h])
        self.final_enc_c = Concatenate()([self.forw_state_c,self.back_state_c])

        # get Context vector
        self.encoder_states =[self.final_enc_h, self.final_enc_c]
    def build_decoder(self):
        self.decoder_inputs = Input(shape=(None,)) 

        # decoder embedding with same number as encoder embedding
        self.dec_emb_layer = Embedding(self.num_decoder_vocab, self.embedding_dim) 
        self.dec_emb = self.dec_emb_layer(self.decoder_inputs)   # apply this way because we need embedding layer for prediction 

        # In encoder we used Bidirectional so it's having two LSTM's so we have to take double units(256*2=512) for single decoder lstm
        # LSTM using encoder's final states as initial state
        self.decoder_lstm = LSTM(self.latent_dim*2, return_sequences=True, return_state=True) 
        self.decoder_outputs, _, _ = self.decoder_lstm(self.dec_emb, initial_state=self.encoder_states)

        # Using Attention Layer
        self.attention_layer = AttentionLayer()
        self.attention_result, self.attention_weights = self.attention_layer([self.encoder_outputs1, self.decoder_outputs])

        # Concat attention output and decoder LSTM output 
        self.decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([self.decoder_outputs, self.attention_result])

        # Dense layer with softmax
        self.decoder_dense = Dense(self.num_decoder_vocab, activation='softmax')
        self.decoder_outputs = self.decoder_dense(self.decoder_concat_input)
        
    def compile(self):
        self.training_model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics = ['acc'])
    
    def fit(self, x_tr, y_tr_in, y_tr_out, x_test, y_test_in, y_test_out, ep, batch_size):
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)
        ck = ModelCheckpoint(filepath='ipa_corrector_best_weights.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')
        Callbacks = [es, ck]
        self.training_model.fit([x_tr,y_tr_in], y_tr_out, epochs = ep, callbacks=Callbacks, batch_size = batch_size, validation_data=(([x_test,y_test_in]), y_test_out))

    def build_inference_model(self):
        self.encoder_model_inference = Model(self.encoder_inputs, outputs = [self.encoder_outputs1, self.final_enc_h, self.final_enc_c])
        self.encoder_model_inference.save('final_encoder_model_ipa_corrector.h5')

        # Decoder Inference
        self.decoder_state_h = Input(shape=(self.latent_dim*2,)) # This numbers has to be same as units of lstm's on which model is trained
        self.decoder_state_c = Input(shape=(self.latent_dim*2,))

        # we need hidden state for attention layer
        # 36 is maximum length if english sentence It has to same as input taken by attention layer can see in model plot
        self.decoder_hidden_state_input = Input(shape=(self.max_encoder_len,self.latent_dim*2)) 
        # get decoder states
        self.dec_states = [self.decoder_state_h, self.decoder_state_c]

        # embedding layer 
        self.dec_emb2 = self.dec_emb_layer(self.decoder_inputs)
        self.decoder_outputs2, self.state_h2, self.state_c2 = self.decoder_lstm(self.dec_emb2, initial_state=self.dec_states)

        # Attention inference
        self.attention_result_inf, self.attention_weights_inf = self.attention_layer([self.decoder_hidden_state_input, self.decoder_outputs2])
        self.decoder_concat_input_inf = Concatenate(axis=-1, name='concat_layer')([self.decoder_outputs2, self.attention_result_inf])

        self.dec_states2= [self.state_h2, self.state_c2]
        self.decoder_outputs2 = self.decoder_dense(self.decoder_concat_input_inf)

        # get decoder model
        self.decoder_model_inference= Model(
                            [self.decoder_inputs] + [self.decoder_hidden_state_input, self.decoder_state_h, self.decoder_state_c],
                             [self.decoder_outputs2]+ self.dec_states2)
        self.decoder_model_inference.save('final_decoder_model_ipa_corrector.h5')
        
    def decode_sequence(self, input_seq, i2o, o2i):
        e_out ,e_h, e_c = self.encoder_model_inference.predict(input_seq, verbose = 0)
        target_seq = np.zeros((1,1))
        target_seq[0,0] = o2i['<']

        stop_condition = False
        decoded_sentence = []

        while not stop_condition:
            (output_tokens, h, c) = self.decoder_model_inference.predict([target_seq] + [e_out, e_h, e_c], verbose = 0)

            # Sample a token
            sampled_token_index = np.argmax(output_tokens[0, -1, :])
            sampled_token = i2o[sampled_token_index]   

            if sampled_token != '>':
                decoded_sentence += [sampled_token]

            # Exit condition: either hit max length or find the stop word.
            if (sampled_token == '>') or (len(decoded_sentence) >= self.max_decoder_len):
                stop_condition = True

            # Update the target sequence (of length 1)
            target_seq = np.zeros((1, 1))
            target_seq[0, 0] = sampled_token_index

            # Update internal states
            (e_h, e_c) = (h, c)
        return decoded_sentence
    def word2seq(self, a2i, input_word):
        final_seq = []
        for c in input_word:
            final_seq += [a2i[c]]
        final_seq = pad_sequences([final_seq], maxlen=self.max_encoder_len, padding='post')[0]
        return final_seq
    
    def translate(self, input_word, a2i, i2o, o2i):
        seq = self.word2seq(a2i, input_word).reshape(1, self.max_encoder_len)
        return self.decode_sequence(seq, i2o, o2i)

In [3]:
# prepare parameters

# fetch metadata
metadata_file = open('metadata_ipa2ipa.txt')
metadata = metadata_file.readlines()
metadata = [line.strip('\n') for line in metadata]
metadata = [line.split('\t') for line in metadata]
metadata = [line[-1] for line in metadata]

max_encoder_len = int(metadata[0])
max_decoder_len = int(metadata[1])
num_encoder_vocab = int(metadata[2])
num_decoder_vocab = int(metadata[3])

# fetch dictionaries
e2i_file = open('ipa2ipa_e2i.pkl', 'rb')
i2e_file = open('ipa2ipa_i2e.pkl', 'rb')
d2i_file = open('ipa2ipa_d2i.pkl', 'rb')
i2d_file = open('ipa2ipa_i2d.pkl', 'rb')

e2i = pickle.load(e2i_file)
i2e = pickle.load(i2e_file)
d2i = pickle.load(d2i_file)
i2d = pickle.load(i2d_file)

e2i_file.close()
i2e_file.close()
d2i_file.close()
i2d_file.close()
metadata_file.close()

In [5]:
# fetch training data
training_data_file = open('training_data_ipa2ipa.txt', encoding = 'UTF-8')
raw_data = training_data_file.readlines()
raw_data = [line.strip('\n') for line in raw_data]
raw_split_data = [line.split('\t')[1:] for line in raw_data]
attempted_list = [pair[0] for pair in raw_split_data]
actual_list = [pair[1] for pair in raw_split_data]

attempted_list = np.array(attempted_list).reshape((1,len(attempted_list)))
actual_list = np.array(actual_list).reshape((1,len(actual_list)))

In [6]:
# create x_train data
x_tr = []

for syl in attempted_list[0]:
    int_seq = []
    for c in syl:
        int_seq += [e2i[c]]
    x_tr += [int_seq]
    
x_tr = pad_sequences(x_tr, maxlen=max_encoder_len, padding='post')

In [7]:
# create y_train data
y_tr = []

for seq in actual_list[0]:
    int_seq = []
    for c in seq:
        int_seq += [d2i[c]]
    y_tr += [int_seq]
y_tr = pad_sequences(y_tr, maxlen=max_decoder_len, padding='post')

In [8]:
#splitting into validation and training

split_index = int(len(x_tr) * .8)

y_test = y_tr[split_index:]
y_test_in = y_test[:, :-1]
y_test_out = y_test[:, 1:]

y_tr = y_tr[:split_index]
y_tr_in = y_tr[:, :-1]
y_tr_out = y_tr[:, 1:]

x_test = x_tr[split_index:]
x_tr = x_tr[:split_index]

In [9]:
print(len(x_tr))

1420


In [10]:
ipa_corrector = bi_model(max_encoder_len, max_decoder_len, num_encoder_vocab, num_decoder_vocab)

In [11]:
ipa_corrector.compile()
ipa_corrector.fit(x_tr, y_tr_in, y_tr_out, x_test, y_test_in, y_test_out, ep=50, batch_size=128)

Epoch 1/50
12/12 [==============================] - 31s 2s/step - loss: 2.3539 - acc: 0.5245 - val_loss: 1.5444 - val_acc: 0.6145

Epoch 00001: val_acc improved from -inf to 0.61453, saving model to ipa_corrector_best_weights.h5
Epoch 2/50
12/12 [==============================] - 23s 2s/step - loss: 1.5144 - acc: 0.6147 - val_loss: 1.4299 - val_acc: 0.6299

Epoch 00002: val_acc improved from 0.61453 to 0.62985, saving model to ipa_corrector_best_weights.h5
Epoch 3/50
12/12 [==============================] - 23s 2s/step - loss: 1.4382 - acc: 0.6297 - val_loss: 1.3571 - val_acc: 0.6320

Epoch 00003: val_acc improved from 0.62985 to 0.63202, saving model to ipa_corrector_best_weights.h5
Epoch 4/50
12/12 [==============================] - 24s 2s/step - loss: 1.3632 - acc: 0.6404 - val_loss: 1.2615 - val_acc: 0.6816

Epoch 00004: val_acc improved from 0.63202 to 0.68156, saving model to ipa_corrector_best_weights.h5
Epoch 5/50
12/12 [==============================] - 24s 2s/step - loss: 1.3

In [20]:
# resume training
ipa_corrector.training_model.load_weights('ipa_corrector_best_weights.h5')
ipa_corrector.fit(x_tr, y_tr_in, y_tr_out, x_test, y_test_in, y_test_out, ep=50, batch_size=128)

Epoch 1/50
12/12 [==============================] - 19s 2s/step - loss: 0.3424 - acc: 0.8980 - val_loss: 0.5014 - val_acc: 0.8492

Epoch 00001: val_acc improved from -inf to 0.84921, saving model to ipa_corrector_best_weights.h5
Epoch 2/50
12/12 [==============================] - 23s 2s/step - loss: 0.3214 - acc: 0.9036 - val_loss: 0.5168 - val_acc: 0.8488

Epoch 00002: val_acc did not improve from 0.84921
Epoch 3/50
12/12 [==============================] - 24s 2s/step - loss: 0.3081 - acc: 0.9095 - val_loss: 0.4891 - val_acc: 0.8530

Epoch 00003: val_acc improved from 0.84921 to 0.85304, saving model to ipa_corrector_best_weights.h5
Epoch 4/50
12/12 [==============================] - 24s 2s/step - loss: 0.2715 - acc: 0.9214 - val_loss: 0.4689 - val_acc: 0.8567

Epoch 00004: val_acc improved from 0.85304 to 0.85674, saving model to ipa_corrector_best_weights.h5
Epoch 5/50
12/12 [==============================] - 23s 2s/step - loss: 0.2473 - acc: 0.9285 - val_loss: 0.4819 - val_acc: 0.8

In [21]:
ipa_corrector.training_model.load_weights('ipa_corrector_best_weights.h5')
ipa_corrector.build_inference_model()

In [26]:
word = attempted_list[0][987]
print(seq2word(word))
print(seq2word(actual_list[0][987]))

print(seq2word(ipa_corrector.translate(word, e2i, i2d, d2i)))

prikɑraɪaʊz
<priˈkɛriəs>
priˈkɔriəs


In [14]:
def seq2word(array):
    final_string = ""
    for c in array:
        final_string += c
    return final_string